<a href="https://colab.research.google.com/github/raf-esc/alura_challenge_data_science_2/blob/main/semana_2/alura_challenge_data_science_2_semana_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alura - Challenge Data Science 2 - Semana 2

## 1ª Tarefa - Preparar o Ambiente e Carregar a Base de Dados

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=8e268dfb1ff40b61730d1beed2886fd586ec027aaec2136107a9a09d55ea82d0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [4]:
spark

In [5]:
# from google.colab import drive
# drive.mount('content/drive')

In [6]:
dados = spark.read.parquet('/content/drive/MyDrive/Alura/Challenge - Data Science 2/dataset_transformado_parquet')

In [7]:
dados.show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|iptu| tipo| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+----+-----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  NULL|       Usado| Apartamento|Residencial|   1|            Realengo|  Zona Oeste|       285|NULL|Venda| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|       Usado| Apartamento|Residencial|   0|               Irajá|  Zona Norte|       170|   0|Vend

In [8]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



## 2ª Tarefa - Seleção de features

A seleção de features é uma etapa muito importante na construção de modelos de Machine Learning. Escolher quais colunas são importantes e descartar as irrelevantes para a análise reduz a complexidade dos dados inseridos e por consequência o modelo irá demorar menos tempo para ser ajustado.

In [9]:
dados.groupBy('tipo_anuncio').count().show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|66562|
+------------+-----+



In [10]:
dados.groupBy('tipo_unidade').count().show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
| Apartamento|66562|
+------------+-----+



In [11]:
dados.groupBy('tipo_uso').count().show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|Residencial|66562|
+-----------+-----+



In [12]:
dados.groupBy('tipo').count().show()

+-----+-----+
| tipo|count|
+-----+-----+
|Venda|66562|
+-----+-----+



In [13]:
df_sem_colunas_com_um_unico_valor = dados.drop('tipo_anuncio').drop('tipo_unidade').drop('tipo_uso').drop('tipo')
df_sem_colunas_com_um_unico_valor.show()

+--------------------+-----+----------+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+----+------+
|                  id|andar|area_total|area_util|banheiros|     caracteristicas|quartos|suites|vaga|              bairro|        zona|condominio|iptu| valor|
+--------------------+-----+----------+---------+---------+--------------------+-------+------+----+--------------------+------------+----------+----+------+
|03a386b6-7ab8-4ef...|    0|        43|       43|        1|[Churrasqueira, A...|      2|  NULL|   1|            Realengo|  Zona Oeste|       285|NULL| 22999|
|1fe78d41-b8e0-4d2...|    0|        44|       44|        1|                  []|      2|     0|   0|               Irajá|  Zona Norte|       170|   0|110000|
|1fa1c1e5-e98c-433...|    4|      NULL|       55|        1|                  []|      2|     0|   1|              Cosmos|  Zona Oeste|      NULL|NULL|115000|
|a6ab01ae-3d40-40e...|    2|      NULL|       55|   

In [14]:
dados.groupBy(f.isnull('area_total')).count().show()

+--------------------+-----+
|(area_total IS NULL)|count|
+--------------------+-----+
|                true| 9194|
|               false|57368|
+--------------------+-----+



In [15]:
dados.groupBy(f.isnull('area_util')).count().show()

+-------------------+-----+
|(area_util IS NULL)|count|
+-------------------+-----+
|              false|66562|
+-------------------+-----+



In [16]:
df_sem_colunas_sem_redundantes = df_sem_colunas_com_um_unico_valor.drop('area_total').drop('bairro')
df_sem_colunas_sem_redundantes.show()

+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+----------+----+------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|        zona|condominio|iptu| valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+----------+----+------+
|03a386b6-7ab8-4ef...|    0|       43|        1|[Churrasqueira, A...|      2|  NULL|   1|  Zona Oeste|       285|NULL| 22999|
|1fe78d41-b8e0-4d2...|    0|       44|        1|                  []|      2|     0|   0|  Zona Norte|       170|   0|110000|
|1fa1c1e5-e98c-433...|    4|       55|        1|                  []|      2|     0|   1|  Zona Oeste|      NULL|NULL|115000|
|a6ab01ae-3d40-40e...|    2|       55|        1|                  []|      2|     0|   0|  Zona Norte|      NULL|NULL|115000|
|1d8ff7a2-16bd-4e3...|    0|       50|        1|                  []|      1|  NULL|   0|Zona Central|      1300|2679|

## 3ª Tarefa - Converter os tipos das colunas

Ao analisarmos a estrutura da nossa base de dados, vamos notar que os tipos dos dados de cada coluna estão incorretos e por isso é necessário convertê-los.

In [17]:
df_sem_colunas_sem_redundantes.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- vaga: long (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- valor: string (nullable = true)



In [18]:
df_com_tipos_corrigidos = df_sem_colunas_sem_redundantes\
.withColumn('andar', df_sem_colunas_sem_redundantes['andar'].cast(IntegerType()))\
.withColumn('area_util', df_sem_colunas_sem_redundantes['area_util'].cast(DoubleType()))\
.withColumn('banheiros', df_sem_colunas_sem_redundantes['banheiros'].cast(IntegerType()))\
.withColumn('quartos', df_sem_colunas_sem_redundantes['quartos'].cast(IntegerType()))\
.withColumn('suites', df_sem_colunas_sem_redundantes['suites'].cast(IntegerType()))\
.withColumn('vaga', df_sem_colunas_sem_redundantes['vaga'].cast(IntegerType()))\
.withColumn('condominio', df_sem_colunas_sem_redundantes['condominio'].cast(DoubleType()))\
.withColumn('iptu', df_sem_colunas_sem_redundantes['iptu'].cast(DoubleType()))\
.withColumn('valor', df_sem_colunas_sem_redundantes['valor'].cast(DoubleType()))

In [19]:
df_com_tipos_corrigidos.show()

+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+----------+------+--------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|        zona|condominio|  iptu|   valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+----------+------+--------+
|03a386b6-7ab8-4ef...|    0|     43.0|        1|[Churrasqueira, A...|      2|  NULL|   1|  Zona Oeste|     285.0|  NULL| 22999.0|
|1fe78d41-b8e0-4d2...|    0|     44.0|        1|                  []|      2|     0|   0|  Zona Norte|     170.0|   0.0|110000.0|
|1fa1c1e5-e98c-433...|    4|     55.0|        1|                  []|      2|     0|   1|  Zona Oeste|      NULL|  NULL|115000.0|
|a6ab01ae-3d40-40e...|    2|     55.0|        1|                  []|      2|     0|   0|  Zona Norte|      NULL|  NULL|115000.0|
|1d8ff7a2-16bd-4e3...|    0|     50.0|        1|                  []|      1|  NULL|   0|Z

In [20]:
df_com_tipos_corrigidos.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)



## 4ª Tarefa - Tratamento da coluna caracteristicas

A coluna caracteristicas possui listas de strings como conteúdo de suas linhas. No entanto, algumas dessas listas estão sem elementos.

Vamos transformar essas listas sem elementos em valores nulos.

In [21]:
df_com_caracteristicas_nulas = df_com_tipos_corrigidos\
.withColumn('caracteristicas',
            f.when(df_com_tipos_corrigidos['caracteristicas'] != f.array(),
                   df_com_tipos_corrigidos['caracteristicas']))

df_com_caracteristicas_nulas.show()

+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+----------+------+--------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|        zona|condominio|  iptu|   valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+----------+------+--------+
|03a386b6-7ab8-4ef...|    0|     43.0|        1|[Churrasqueira, A...|      2|  NULL|   1|  Zona Oeste|     285.0|  NULL| 22999.0|
|1fe78d41-b8e0-4d2...|    0|     44.0|        1|                NULL|      2|     0|   0|  Zona Norte|     170.0|   0.0|110000.0|
|1fa1c1e5-e98c-433...|    4|     55.0|        1|                NULL|      2|     0|   1|  Zona Oeste|      NULL|  NULL|115000.0|
|a6ab01ae-3d40-40e...|    2|     55.0|        1|                NULL|      2|     0|   0|  Zona Norte|      NULL|  NULL|115000.0|
|1d8ff7a2-16bd-4e3...|    0|     50.0|        1|                NULL|      1|  NULL|   0|Z

## 5ª Tarefa - Tratar dados nulos e NaN's

Ao analisarmos a estrutura da nossa base de dados, vamos notar que ela possui diversos dados faltantes. Dessa maneira, é importante lidarmos com isso, pois modelos de Machine Learning não costumam trabalhar corretamente com dados nulos.

In [22]:
df_nulos = df_com_caracteristicas_nulas\
.select([f.isnull(column).alias(f'{column}_nulos') for column in df_com_caracteristicas_nulas.columns])

for column in df_nulos.columns:
  df_nulos.groupBy(column).count().show()

+--------+-----+
|id_nulos|count|
+--------+-----+
|   false|66562|
+--------+-----+

+-----------+-----+
|andar_nulos|count|
+-----------+-----+
|      false|66562|
+-----------+-----+

+---------------+-----+
|area_util_nulos|count|
+---------------+-----+
|          false|66562|
+---------------+-----+

+---------------+-----+
|banheiros_nulos|count|
+---------------+-----+
|          false|66562|
+---------------+-----+

+---------------------+-----+
|caracteristicas_nulos|count|
+---------------------+-----+
|                 true|12736|
|                false|53826|
+---------------------+-----+

+-------------+-----+
|quartos_nulos|count|
+-------------+-----+
|        false|66562|
+-------------+-----+

+------------+-----+
|suites_nulos|count|
+------------+-----+
|        true| 5554|
|       false|61008|
+------------+-----+

+----------+-----+
|vaga_nulos|count|
+----------+-----+
|      true| 3017|
|     false|63545|
+----------+-----+

+----------+-----+
|zona_nulos|count|

In [23]:
df_com_nulos_tratados = df_com_caracteristicas_nulas.fillna({'suites': 0, 'vaga': 0, 'condominio': 0,'iptu': 0})

In [24]:
df_com_nulos_tratados.show()

+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+----------+------+--------+
|                  id|andar|area_util|banheiros|     caracteristicas|quartos|suites|vaga|        zona|condominio|  iptu|   valor|
+--------------------+-----+---------+---------+--------------------+-------+------+----+------------+----------+------+--------+
|03a386b6-7ab8-4ef...|    0|     43.0|        1|[Churrasqueira, A...|      2|     0|   1|  Zona Oeste|     285.0|   0.0| 22999.0|
|1fe78d41-b8e0-4d2...|    0|     44.0|        1|                NULL|      2|     0|   0|  Zona Norte|     170.0|   0.0|110000.0|
|1fa1c1e5-e98c-433...|    4|     55.0|        1|                NULL|      2|     0|   1|  Zona Oeste|       0.0|   0.0|115000.0|
|a6ab01ae-3d40-40e...|    2|     55.0|        1|                NULL|      2|     0|   0|  Zona Norte|       0.0|   0.0|115000.0|
|1d8ff7a2-16bd-4e3...|    0|     50.0|        1|                NULL|      1|     0|   0|Z

## 6ª Tarefa - Preparação dos dados para machine learning

Para conseguirmos utilizar nossos dados em modelos de Machine Learning, é necessário realizarmos algumas técnicas, como a transformação de variáveis categóricas em binárias.

In [25]:
zona_dummies = df_com_nulos_tratados\
.groupBy('id')\
.pivot('zona')\
.count()\
.fillna(0)\
.drop('')

zona_dummies.show()

+--------------------+------------+----------+----------+--------+
|                  id|Zona Central|Zona Norte|Zona Oeste|Zona Sul|
+--------------------+------------+----------+----------+--------+
|4e47e4d4-3326-4eb...|           0|         0|         0|       1|
|02fba6ef-a691-442...|           0|         0|         1|       0|
|fc03c1a9-8bbb-41a...|           0|         1|         0|       0|
|3dd5d200-0a7f-43d...|           0|         0|         0|       1|
|82707939-71bd-40c...|           0|         0|         0|       1|
|ece1e671-cb4d-48a...|           1|         0|         0|       0|
|3521e03b-0815-4aa...|           0|         0|         0|       1|
|03a386b6-7ab8-4ef...|           0|         0|         1|       0|
|107507e0-f075-47e...|           0|         0|         1|       0|
|e5e00255-6408-4e6...|           0|         0|         1|       0|
|c7e56e3a-a2b2-48a...|           0|         0|         1|       0|
|b37a2a08-d3c6-489...|           0|         1|         0|     

In [26]:
caracteristicas_dummies = df_com_nulos_tratados\
.select('id', f.explode('caracteristicas').alias('caracteristicas'))\
.groupBy('id')\
.pivot('caracteristicas')\
.count()\
.fillna(0)

caracteristicas_dummies.show()

+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|fd96bbd5-d631-416...|       1|                 1|            1|                 1|       1|      1|         1|           1|                1|              1|
|bfffedfe-99e7-4ae...|       0|                 1|            1|                 1|       1|      0|         1|           0|                1|              1|
|fcb67af3-5601-415...|       1|                 1|            1|                 0|       0|      1|         1|           0|                0|              1|
|afecddff-f4cc-4ab...|       1|               

In [27]:
df_com_dummies = df_com_nulos_tratados\
.join(zona_dummies, on = 'id')\
.join(caracteristicas_dummies, on = 'id', how = 'outer')\
.drop('caracteristicas')\
.drop('zona')\
.fillna(0)

df_com_dummies.show()

+--------------------+-----+---------+---------+-------+------+----+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|condominio|  iptu|    valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00012605-9cae-45b...|    0|    410.0|        3|      4|     1|   1|    3300.0|5780.0|4600000.0|           0|         0|         0|       1|       0|                 0| 

In [28]:
df_nulos_2 = df_com_dummies\
.select([f.isnull(column).alias(f'{column}_nulos') for column in df_com_dummies.columns])

for column in df_nulos_2.columns:
  df_nulos_2.groupBy(column).count().show()

+--------+-----+
|id_nulos|count|
+--------+-----+
|   false|66562|
+--------+-----+

+-----------+-----+
|andar_nulos|count|
+-----------+-----+
|      false|66562|
+-----------+-----+

+---------------+-----+
|area_util_nulos|count|
+---------------+-----+
|          false|66562|
+---------------+-----+

+---------------+-----+
|banheiros_nulos|count|
+---------------+-----+
|          false|66562|
+---------------+-----+

+-------------+-----+
|quartos_nulos|count|
+-------------+-----+
|        false|66562|
+-------------+-----+

+------------+-----+
|suites_nulos|count|
+------------+-----+
|       false|66562|
+------------+-----+

+----------+-----+
|vaga_nulos|count|
+----------+-----+
|     false|66562|
+----------+-----+

+----------------+-----+
|condominio_nulos|count|
+----------------+-----+
|           false|66562|
+----------------+-----+

+----------+-----+
|iptu_nulos|count|
+----------+-----+
|     false|66562|
+----------+-----+

+-----------+-----+
|valor_nulos|cou

## 7ª Tarefa - Salvar a base de dados em parquet

Nesse projeto, criamos uma base dados ideal para utilizar em modelos de machine learning. É importante exportar ela para conseguir utilizar em outros modelos e para treinar novamente nosso modelo, caso seja necessário.

In [29]:
df_com_dummies.write\
.parquet('/content/drive/MyDrive/Alura/Challenge - Data Science 2/parquet_para_regressao', mode = 'overwrite')

## 8ª Tarefa - Preparação dos dados para os algoritmos do Spark MLlib

Para utilizarmos nossos dados em modelos do PySpark, nós precisamos utilizar a técnica de vetorização dos dados.

In [30]:
dataset = df_com_dummies.withColumnRenamed('valor', 'label')

dataset.show()

+--------------------+-----+---------+---------+-------+------+----+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|condominio|  iptu|    label|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00012605-9cae-45b...|    0|    410.0|        3|      4|     1|   1|    3300.0|5780.0|4600000.0|           0|         0|         0|       1|       0|                 0| 

In [31]:
lista_features = dataset.drop('id').drop('label').columns

lista_features

['andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'condominio',
 'iptu',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

In [32]:
assembler = VectorAssembler(inputCols = lista_features, outputCol = 'features')

In [33]:
dataset_pronto = assembler.transform(dataset).select('features', 'label')

dataset_pronto.show(truncate = False)

+-------------------------------------------------------------------------------------------------+---------+
|features                                                                                         |label    |
+-------------------------------------------------------------------------------------------------+---------+
|(22,[1,2,3,4,5,6,7,11,16],[410.0,3.0,4.0,1.0,1.0,3300.0,5780.0,1.0,1.0])                         |4600000.0|
|(22,[0,1,2,3,5,6,7,9,13,16,20,21],[1.0,50.0,1.0,1.0,1.0,500.0,600.0,1.0,1.0,1.0,1.0,1.0])        |360000.0 |
|(22,[1,2,3,4,5,10,14,16,17],[148.0,4.0,4.0,2.0,2.0,1.0,1.0,1.0,1.0])                             |1200000.0|
|(22,[1,2,3,6,7,11,16],[104.0,1.0,2.0,1010.0,272.0,1.0,1.0])                                      |750000.0 |
|(22,[1,2,3,5,6,7,11,16],[71.0,2.0,2.0,1.0,1312.0,360.0,1.0,1.0])                                 |1025000.0|
|[3.0,115.0,2.0,3.0,0.0,0.0,1904.0,3831.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0]|1100000.0|
|[0.0,65.0

## 9ª Tarefa - Criação dos modelos de regressão e
## 10ª Tarefa - Avaliação dos modelos

In [34]:
treino, teste = dataset_pronto.randomSplit([0.8, 0.2], seed = 1999)

In [35]:
print(f'Itens do dataset inicial: {dataset_pronto.count()}.')
print(f'Itens do dataset de treino: {treino.count()}.')
print(f'Itens do dataset de teste: {teste.count()}.')

Itens do dataset inicial: 66562.
Itens do dataset de treino: 53261.
Itens do dataset de teste: 13301.


### 1º Modelo: Regressão Linear

In [36]:
objeto_lr = LinearRegression()

In [37]:
modelo_lr = objeto_lr.fit(treino)

In [38]:
previsoes_lr_treino = modelo_lr.transform(treino)
previsoes_lr_treino.show()

+--------------------+---------+------------------+
|            features|    label|        prediction|
+--------------------+---------+------------------+
|(22,[0,1,2,3,4,5,...| 445000.0| 654240.4234794453|
|(22,[0,1,2,3,4,5,...| 539000.0| 609932.9219434478|
|(22,[0,1,2,3,4,5,...| 262000.0| 754104.1545912768|
|(22,[0,1,2,3,4,5,...| 460000.0| 222501.7969483217|
|(22,[0,1,2,3,4,5,...| 349000.0| 375961.1324242596|
|(22,[0,1,2,3,4,5,...| 400000.0|386276.21063899266|
|(22,[0,1,2,3,4,5,...| 540000.0| 525937.1065626056|
|(22,[0,1,2,3,4,5,...|1047795.0| 672557.5341689175|
|(22,[0,1,2,3,4,5,...| 750000.0| 785338.5051128364|
|(22,[0,1,2,3,4,5,...| 795000.0|1003886.2012783908|
|(22,[0,1,2,3,4,5,...| 391000.0| 314503.4085505609|
|(22,[0,1,2,3,4,5,...| 473400.0| 447685.2761466368|
|(22,[0,1,2,3,4,5,...| 659900.0|457728.69041384873|
|(22,[0,1,2,3,4,5,...| 797204.0| 498650.1995176454|
|(22,[0,1,2,3,4,5,...| 550000.0|1106049.0881770335|
|(22,[0,1,2,3,4,5,...|1400000.0|2078272.7939828571|
|(22,[0,1,2,

In [39]:
previsoes_lr_teste = modelo_lr.transform(teste)
previsoes_lr_teste.show()

+--------------------+---------+------------------+
|            features|    label|        prediction|
+--------------------+---------+------------------+
|(22,[0,1,2,3,4,5,...| 318000.0| 580561.3930812776|
|(22,[0,1,2,3,4,5,...| 335000.0| 638642.6026340767|
|(22,[0,1,2,3,4,5,...| 499000.0| 740966.3434011247|
|(22,[0,1,2,3,4,5,...| 950000.0| 1443658.334489614|
|(22,[0,1,2,3,4,5,...|2450000.0| 4293403.839034559|
|(22,[0,1,2,3,4,5,...| 299000.0| 147543.8798939996|
|(22,[0,1,2,3,4,5,...| 530000.0| 416988.0592140515|
|(22,[0,1,2,3,4,5,...| 750000.0| 860332.7939631564|
|(22,[0,1,2,3,4,5,...|1340079.0| 1324135.929979673|
|(22,[0,1,2,3,4,5,...| 590000.0| 570472.7235159341|
|(22,[0,1,2,3,4,5,...| 344000.0|  376168.002269039|
|(22,[0,1,2,3,4,5,...| 580000.0| 955826.3340165261|
|(22,[0,1,2,3,4,5,...|1350000.0|1744004.7565104747|
|(22,[0,1,2,3,4,5,...| 730000.0|1063838.1393616383|
|(22,[0,1,2,3,4,5,...| 816000.0| 741293.9029489117|
|(22,[0,1,2,3,4,5,...| 420000.0|303180.62538341095|
|(22,[0,1,2,

In [40]:
resumo_treino_lr = modelo_lr.summary
resumo_teste_lr = modelo_lr.evaluate(teste)

In [41]:
separador = '-' * 45

print(f'''{separador}
Relatório do modelo - regressão linear.
{separador}
Resultados do treino:
R²: {resumo_treino_lr.r2}.
RMSE: {resumo_treino_lr.rootMeanSquaredError}
{separador}
Resultados do teste:
R²: {resumo_teste_lr.r2}.
RMSE: {resumo_teste_lr.rootMeanSquaredError}
{separador}''')

---------------------------------------------
Relatório do modelo - regressão linear.
---------------------------------------------
Resultados do treino:
R²: 0.6731037630813469.
RMSE: 840313.8170974319
---------------------------------------------
Resultados do teste:
R²: 0.6349382623115064.
RMSE: 893983.7772467382
---------------------------------------------


### 2º Modelo: Árvore de Decisão

In [42]:
objeto_dtr = DecisionTreeRegressor(maxDepth = 10, seed = 1999)

In [43]:
modelo_dtr = objeto_dtr.fit(treino)

In [44]:
previsoes_dtr_treino = modelo_dtr.transform(treino)
previsoes_dtr_treino.show()

+--------------------+---------+------------------+
|            features|    label|        prediction|
+--------------------+---------+------------------+
|(22,[0,1,2,3,4,5,...| 445000.0|422178.49436936935|
|(22,[0,1,2,3,4,5,...| 539000.0|422178.49436936935|
|(22,[0,1,2,3,4,5,...| 262000.0| 553344.3318181818|
|(22,[0,1,2,3,4,5,...| 460000.0|310998.42050691246|
|(22,[0,1,2,3,4,5,...| 349000.0| 359922.6626429479|
|(22,[0,1,2,3,4,5,...| 400000.0| 520129.9196787149|
|(22,[0,1,2,3,4,5,...| 540000.0| 642766.1532663316|
|(22,[0,1,2,3,4,5,...|1047795.0| 760952.4013245034|
|(22,[0,1,2,3,4,5,...| 750000.0| 673250.8997214484|
|(22,[0,1,2,3,4,5,...| 795000.0| 673250.8997214484|
|(22,[0,1,2,3,4,5,...| 391000.0| 359922.6626429479|
|(22,[0,1,2,3,4,5,...| 473400.0|422178.49436936935|
|(22,[0,1,2,3,4,5,...| 659900.0| 596919.8050314465|
|(22,[0,1,2,3,4,5,...| 797204.0| 760952.4013245034|
|(22,[0,1,2,3,4,5,...| 550000.0| 673250.8997214484|
|(22,[0,1,2,3,4,5,...|1400000.0| 1560039.298245614|
|(22,[0,1,2,

In [45]:
previsoes_dtr_teste = modelo_dtr.transform(teste)
previsoes_dtr_teste.show()

+--------------------+---------+------------------+
|            features|    label|        prediction|
+--------------------+---------+------------------+
|(22,[0,1,2,3,4,5,...| 318000.0| 458334.7622807018|
|(22,[0,1,2,3,4,5,...| 335000.0| 553404.7272727273|
|(22,[0,1,2,3,4,5,...| 499000.0| 673250.8997214484|
|(22,[0,1,2,3,4,5,...| 950000.0|      1307831.3875|
|(22,[0,1,2,3,4,5,...|2450000.0|3385533.3333333335|
|(22,[0,1,2,3,4,5,...| 299000.0|310998.42050691246|
|(22,[0,1,2,3,4,5,...| 530000.0| 458334.7622807018|
|(22,[0,1,2,3,4,5,...| 750000.0| 673250.8997214484|
|(22,[0,1,2,3,4,5,...|1340079.0|1053801.5757575757|
|(22,[0,1,2,3,4,5,...| 590000.0| 553404.7272727273|
|(22,[0,1,2,3,4,5,...| 344000.0| 359922.6626429479|
|(22,[0,1,2,3,4,5,...| 580000.0| 673250.8997214484|
|(22,[0,1,2,3,4,5,...|1350000.0|1901823.2544169612|
|(22,[0,1,2,3,4,5,...| 730000.0| 988380.7961038961|
|(22,[0,1,2,3,4,5,...| 816000.0| 642766.1532663316|
|(22,[0,1,2,3,4,5,...| 420000.0|422178.49436936935|
|(22,[0,1,2,

In [46]:
evaluator = RegressionEvaluator()

In [47]:
print(f'''{separador}
Relatório do modelo - árvore de decisão.
{separador}
Resultados do treino:
R²: {evaluator.evaluate(previsoes_dtr_treino, {evaluator.metricName: 'r2'})}.
RMSE: {evaluator.evaluate(previsoes_dtr_treino, {evaluator.metricName: 'rmse'})}
{separador}
Resultados do teste:
R²: {evaluator.evaluate(previsoes_dtr_teste, {evaluator.metricName: 'r2'})}.
RMSE: {evaluator.evaluate(previsoes_dtr_teste, {evaluator.metricName: 'rmse'})}
{separador}''')

---------------------------------------------
Relatório do modelo - árvore de decisão.
---------------------------------------------
Resultados do treino:
R²: 0.8624081708853881.
RMSE: 545171.1509904137
---------------------------------------------
Resultados do teste:
R²: 0.8022925293857479.
RMSE: 657897.1719509996
---------------------------------------------


### 3º Modelo: Floresta Aleatória

In [48]:
objeto_rfr = RandomForestRegressor(maxDepth = 10, numTrees = 14, seed = 1999)

In [49]:
modelo_rfr = objeto_rfr.fit(treino)

In [50]:
previsoes_rfr_treino = modelo_rfr.transform(treino)
previsoes_rfr_treino.show()

+--------------------+---------+------------------+
|            features|    label|        prediction|
+--------------------+---------+------------------+
|(22,[0,1,2,3,4,5,...| 445000.0|394610.67856197903|
|(22,[0,1,2,3,4,5,...| 539000.0| 417756.0748121693|
|(22,[0,1,2,3,4,5,...| 262000.0|498919.95406971464|
|(22,[0,1,2,3,4,5,...| 460000.0| 375193.3372217367|
|(22,[0,1,2,3,4,5,...| 349000.0| 384413.5847655248|
|(22,[0,1,2,3,4,5,...| 400000.0| 430710.9024019449|
|(22,[0,1,2,3,4,5,...| 540000.0| 583378.9288092984|
|(22,[0,1,2,3,4,5,...|1047795.0| 615907.1387457177|
|(22,[0,1,2,3,4,5,...| 750000.0| 679383.3273476355|
|(22,[0,1,2,3,4,5,...| 795000.0| 783881.4092763781|
|(22,[0,1,2,3,4,5,...| 391000.0| 405115.3601175099|
|(22,[0,1,2,3,4,5,...| 473400.0| 477782.9673842507|
|(22,[0,1,2,3,4,5,...| 659900.0| 544940.2253916911|
|(22,[0,1,2,3,4,5,...| 797204.0| 573295.7938363433|
|(22,[0,1,2,3,4,5,...| 550000.0| 710501.7689410577|
|(22,[0,1,2,3,4,5,...|1400000.0|1234548.6147872612|
|(22,[0,1,2,

In [51]:
previsoes_rfr_teste = modelo_rfr.transform(teste)
previsoes_rfr_teste.show()

+--------------------+---------+------------------+
|            features|    label|        prediction|
+--------------------+---------+------------------+
|(22,[0,1,2,3,4,5,...| 318000.0|517657.20024853747|
|(22,[0,1,2,3,4,5,...| 335000.0|  573697.162423917|
|(22,[0,1,2,3,4,5,...| 499000.0| 637842.4584479714|
|(22,[0,1,2,3,4,5,...| 950000.0|1167595.0383304427|
|(22,[0,1,2,3,4,5,...|2450000.0|2971286.1926755495|
|(22,[0,1,2,3,4,5,...| 299000.0|349945.03189820127|
|(22,[0,1,2,3,4,5,...| 530000.0|430649.73759648745|
|(22,[0,1,2,3,4,5,...| 750000.0| 685372.8416881458|
|(22,[0,1,2,3,4,5,...|1340079.0| 1070728.544720754|
|(22,[0,1,2,3,4,5,...| 590000.0| 558086.1283167732|
|(22,[0,1,2,3,4,5,...| 344000.0|401365.29877562507|
|(22,[0,1,2,3,4,5,...| 580000.0| 654970.8117687985|
|(22,[0,1,2,3,4,5,...|1350000.0|1536964.5371006876|
|(22,[0,1,2,3,4,5,...| 730000.0|  815388.977929428|
|(22,[0,1,2,3,4,5,...| 816000.0| 612756.8498711511|
|(22,[0,1,2,3,4,5,...| 420000.0| 411506.8338743538|
|(22,[0,1,2,

In [52]:
print(f'''{separador}
Relatório do modelo - floresta aleatória.
{separador}
Resultados do treino:
R²: {evaluator.evaluate(previsoes_rfr_treino, {evaluator.metricName: 'r2'})}.
RMSE: {evaluator.evaluate(previsoes_rfr_treino, {evaluator.metricName: 'rmse'})}
{separador}
Resultados do teste:
R²: {evaluator.evaluate(previsoes_rfr_teste, {evaluator.metricName: 'r2'})}.
RMSE: {evaluator.evaluate(previsoes_rfr_teste, {evaluator.metricName: 'rmse'})}
{separador}''')

---------------------------------------------
Relatório do modelo - floresta aleatória.
---------------------------------------------
Resultados do treino:
R²: 0.8656100615413697.
RMSE: 538790.4899971351
---------------------------------------------
Resultados do teste:
R²: 0.8322141660070951.
RMSE: 606071.8668411385
---------------------------------------------


In [53]:
print(f'''{separador}
Relatório comparativo de modelos.
{separador}
Resultados do treino.
{separador}
R²
Regressão linear: {resumo_treino_lr.r2}.
Árvore de decisão: {evaluator.evaluate(previsoes_dtr_treino, {evaluator.metricName: 'r2'})}.
Floresta aleatória: {evaluator.evaluate(previsoes_rfr_treino, {evaluator.metricName: 'r2'})}.
{separador}
RMSE
Regressão linear: {resumo_treino_lr.rootMeanSquaredError}
Árvore de decisão: {evaluator.evaluate(previsoes_dtr_treino, {evaluator.metricName: 'rmse'})}.
Floresta aleatória: {evaluator.evaluate(previsoes_rfr_treino, {evaluator.metricName: 'rmse'})}.
{separador}
Resultados do teste.
{separador}
R²
Regressão linear: {resumo_teste_lr.r2}.
Árvore de decisão: {evaluator.evaluate(previsoes_dtr_teste, {evaluator.metricName: 'r2'})}.
Floresta aleatória: {evaluator.evaluate(previsoes_rfr_teste, {evaluator.metricName: 'r2'})}.
{separador}
RMSE
Regressão linear: {resumo_teste_lr.rootMeanSquaredError}.
Árvore de decisão: {evaluator.evaluate(previsoes_dtr_teste, {evaluator.metricName: 'rmse'})}.
Floresta aleatória: {evaluator.evaluate(previsoes_rfr_teste, {evaluator.metricName: 'rmse'})}.
{separador}''')

---------------------------------------------
Relatório comparativo de modelos.
---------------------------------------------
Resultados do treino.
---------------------------------------------
R²
Regressão linear: 0.6731037630813469.
Árvore de decisão: 0.8624081708853881.
Floresta aleatória: 0.8656100615413697.
---------------------------------------------
RMSE
Regressão linear: 840313.8170974319
Árvore de decisão: 545171.1509904137.
Floresta aleatória: 538790.4899971351.
---------------------------------------------
Resultados do teste.
---------------------------------------------
R²
Regressão linear: 0.6349382623115064.
Árvore de decisão: 0.8022925293857479.
Floresta aleatória: 0.8322141660070951.
---------------------------------------------
RMSE
Regressão linear: 893983.7772467382.
Árvore de decisão: 657897.1719509996.
Floresta aleatória: 606071.8668411385.
---------------------------------------------
